In [3]:
import os 
from dotenv import load_dotenv
from langchain.memory import ConversationBufferWindowMemory
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_openai import ChatOpenAI
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
load_dotenv()

True

In [4]:

import PyPDF2

# Open the PDF file in read binary mode
with open('C:/Users/Hetansh_Shah/OneDrive - Dell Technologies/Documents/Dependancy_Finder/thinos-2402-admin-guide.pdf', 'rb') as file:
    # Create a PDF reader object
    pdf_reader = PyPDF2.PdfReader(file)

    # Initialize an empty string to store the extracted text
    text = ''

    # Iterate over each page in the PDF
    for page in pdf_reader.pages:
        # Extract the text from the page and append it to the 'text' variable
        text += page.extract_text()

# Print the extracted text
input = text[:100000]

In [5]:
from openai import OpenAI
import httpx
import os

# As mentioned in the above Note try to setup Dell certificate in your environment to avoid verify=False(SSL verification is disabled).
http_client=httpx.Client(verify=False)
client = OpenAI(
    base_url='https://opensource-challenger-api.prdlvgpu1.aiaccel.dell.com/v1',
    http_client=http_client,
    api_key=os.environ['LLM_API']
)
streaming = True
max_output_tokens = 100

# Read the content of 'output.txt' as the required prompt
# with open('../output.txt', 'r') as f:
#     required_prompt = f.read()

# Available Models list
available_models = ["mixtral-8x7b-instruct-v01", "llamaguard-7b", "gemma-7b-it", "mistral-7b-instruct-v02", "phi-2", "llama-2-70b-chat", "phi-3-mini-128k-instruct", "llama-3-8b-instruct"]

# Let's select the model from the available list
model_selected = available_models[6]


In [6]:
with open('../output2.txt', 'r') as f:
    required_prompt = f.read()
len(required_prompt)

201481

In [8]:
import re
def parse_output_file(filepath):
    text =''
    paths,contents = [],[]
    with open(filepath, 'r') as f:
        text = f.read()
    matches = re.finditer(r'C:/.*',text)
    matches1 = [match for match in matches]# make this the last section of the codebase
    for match in matches1:
        paths.append(match.group(0))
    for match, match_next in zip(matches1[:-1],matches1[1:]):
        ending_index = match.span()[1]
        starting_index = match_next.span()[0]
        contents.append(text[ending_index:starting_index])
    
    #appending the last file
    contents.append(text[matches1[-1].span()[1]:])
    return paths,contents

paths,contents =parse_output_file("../output2.txt")
print(len(paths), len(contents))

44 44


In [15]:
import re
from typing import List, Tuple

def chunk_code(content: str, max_chunk_size: int = 2048) -> List[str]:
    """
    Split the code into context-aware chunks.
    """
    # Split the content into lines
    lines = content.split('\n')
    chunks = []
    current_chunk = []
    current_chunk_size = 0

    for line in lines:
        line_size = len(line) + 1  # +1 for the newline character
        if current_chunk_size + line_size > max_chunk_size and current_chunk:
            chunks.append('\n'.join(current_chunk))
            current_chunk = []
            current_chunk_size = 0

        current_chunk.append(line)
        current_chunk_size += line_size

    if current_chunk:
        chunks.append('\n'.join(current_chunk))

    return chunks

def generate_analysis(contents: List[str], paths: List[str]):
    for content, path in zip(contents, paths):
        path_parts = path.split('\\')[1:]
        path = '/'.join(path_parts)

        # Perform context-aware chunking
        chunks = chunk_code(content)

        for i, chunk in enumerate(chunks):
            required_prompt = f'''<|user|> {input}



             Tell me about Firmware upgrades <|end|><|assistant|>'''

            # Generate test script using the custom LLM
            test_script =  client.completions.create(
                model=model_selected,
                max_tokens=1000,
                prompt=required_prompt,
                stream=False
            ).choices[0].text
            print(f"Test script for {path} (chunk {i+1} of {len(chunks)}):")
            print(test_script)
            print("----------------------------------------------------------------")

In [12]:
required_prompt = f'''<|user|> 
                You are a admin helper

                {input}



             Tell me about BIOS settings pointwise <|end|><|assistant|>'''

test_script =  client.completions.create(
                model=model_selected,
                max_tokens=350,
                prompt=required_prompt,
                stream=True
            )

print(type(test_script))
print("----------------------------------------------------------------")

<class 'openai.Stream'>
----------------------------------------------------------------


In [16]:
generate_analysis(contents,paths)

Test script for library_app/src/main/java/com/example/library_app/repository/BookRepository.java (chunk 1 of 1):
 The provided document is the Administrator's Guide for Dell ThinOS 2402 and 2405, which includes detailed chapters on what's new in each version, how to upgrade firmware, manage settings, troubleshoot issues, and use system tools effectively. It also outlines the supported platforms, functionalities, and configuration steps using Wyse Management Suite for optimal performance and security. Key points include secure DHCP option tags for registration, USB imaging tools for firmware upgrade, and a comprehensive list of ThinOS packages for various updates and optimizations. The document emphasizes the need for proper procedures for registering devices, upgrading, and managing updates to ensure smooth functionality and security in ThinOS environments.
----------------------------------------------------------------
Test script for library_app/build/resources/main/static/upload.ht

In [ ]:


print(f"Model: {model_selected}")
completion = client.completions.create(
    model=model_selected,
    max_tokens=max_output_tokens,
    prompt="Hi, I am bob, tell me about yourself.",
    stream=streaming
)

if streaming:
    for chunk in completion:
        print(chunk.choices[0].text, end='')
else:
    print(completion.choices[0].text)


Model: mixtral-8x7b-instruct-v01

Hi Bob, I am a bot, I analyze input and help find ways to solve problems. How can I help you today?

Recently, there's been a seemingly endless number of generative AI projects, from "fine-tuned" public models to fully proprietary closed systems. These models help us do everything from drafting emails to creating images from textual descriptions, but they often lack the ability to control the tone or style of output.

I

In [12]:
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_openai import ChatOpenAI


store = {}  # memory is maintained outside the chain

def get_session_history(session_id: str) -> InMemoryChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

llm = ChatOpenAI(
    base_url='https://opensource-challenger-api.prdlvgpu1.aiaccel.dell.com/v1',
    model=model_selected,
    openai_api_key=os.environ['LLM_API'],
    http_client=http_client
)

chain = RunnableWithMessageHistory(llm, get_session_history)
chain.invoke(
    "Hi I'm Bob.",
    config={"configurable": {"session_id": "1"}},
)  # session_id determines thread

Parent run 7f4a3b7a-c43e-4d4d-8d02-32a536c98eae not found for run b5e7127d-d482-4809-af11-4fa954e8ebde. Treating as a root run.


AIMessage(content=" Hello Bob! It's nice to meet you. How can I help you today? Is there something you would like to know or talk about? I'm here to answer any questions you have to the best of my ability.", response_metadata={'token_usage': {'completion_tokens': 48, 'prompt_tokens': 14, 'total_tokens': 62}, 'model_name': 'mixtral-8x7b-instruct-v01', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-b5e7127d-d482-4809-af11-4fa954e8ebde-0', usage_metadata={'input_tokens': 14, 'output_tokens': 48, 'total_tokens': 62})

In [14]:



store = {}  # memory is maintained outside the chain

def get_session_history(session_id: str) -> InMemoryChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
        return store[session_id]

    memory = ConversationBufferWindowMemory(
        chat_memory=store[session_id],
        k=3,
        return_messages=True,
    )
    assert len(memory.memory_variables) == 1
    key = memory.memory_variables[0]
    messages = memory.load_memory_variables({})[key]
    store[session_id] = InMemoryChatMessageHistory(messages=messages)
    return store[session_id]

llm = ChatOpenAI(base_url='https://opensource-challenger-api.prdlvgpu1.aiaccel.dell.com/v1',
    model=model_selected,
    openai_api_key=os.environ['LLM_API'],
    http_client=http_client)

chain = RunnableWithMessageHistory(llm, get_session_history)
chain.invoke(
    "Hi I'm Bob.",
    config={"configurable": {"session_id": "1"}},
)  # session_id determines thread

Parent run 01dee4e9-428e-4a98-9585-bec7cf7ef13b not found for run fb4faafb-4435-4391-aa85-d8c13d9b8e3b. Treating as a root run.


AIMessage(content=" Hello Bob! It's nice to meet you. Is there something specific you would like to talk about or ask me? I'm here to help with any questions you have to the best of my ability.", response_metadata={'token_usage': {'completion_tokens': 44, 'prompt_tokens': 15, 'total_tokens': 59}, 'model_name': 'mixtral-8x7b-instruct-v01', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-fb4faafb-4435-4391-aa85-d8c13d9b8e3b-0', usage_metadata={'input_tokens': 15, 'output_tokens': 44, 'total_tokens': 59})

In [5]:
print(f"Model: {model_selected}")

# Chat history management
store = {}  # memory is maintained outside the chain

def get_session_history(session_id: str) -> InMemoryChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

def print_chat_history(session_id: str):
    history = get_session_history(session_id)
    print(f"\nChat History for session {session_id}:")
    for message in history.messages:
        print(f"{message.type}: {message.content}")

# Create a proper LLM object
llm = ChatOpenAI(
    base_url='https://opensource-challenger-api.prdlvgpu1.aiaccel.dell.com/v1',
    model=model_selected,
    openai_api_key=os.environ['LLM_API'],
    http_client=http_client
)

chain = RunnableWithMessageHistory(llm, get_session_history)

# Check initial state
print_chat_history("1")

# First interaction
print("\nFirst interaction:")
response1 = chain.invoke(
    "Hi, I'm Bob. What's your name?",
    config={"configurable": {"session_id": "1"}},
)
print("Assistant:", response1.content)

# Check history after first interaction
print_chat_history("1")

# Second interaction
print("\nSecond interaction:")
response2 = chain.invoke(
    "Nice to meet you! What do you know about Python?",
    config={"configurable": {"session_id": "1"}},
)
print("Assistant:", response2.content)

# Check history after second interaction
print_chat_history("1")

# Third interaction with a different session
print("\nThird interaction (different session):")
response3 = chain.invoke(
    "Hello, I'm Alice. Can you tell me about the weather?",
    config={"configurable": {"session_id": "2"}},
)
print("Assistant:", response3.content)

# Check history for both sessions
print_chat_history("1")
print_chat_history("2")

Model: mixtral-8x7b-instruct-v01


Parent run 18e5518c-27c9-43b1-a16d-2347b38ae88c not found for run 910a9025-6d3f-48eb-8df0-cdecb7af0cd8. Treating as a root run.



Chat History for session 1:

First interaction:


Parent run 82dc0031-c33e-4ca0-8e4d-cd2fe3dab462 not found for run f8978762-d9b5-4cdc-b2c0-9f3dff6b7a82. Treating as a root run.


Assistant:  Hello Bob! I'm an assistant. I don't have a personal name as I'm a computer program, but you can call me Assistant. How can I assist you today?

Chat History for session 1:
human: Hi, I'm Bob. What's your name?
ai:  Hello Bob! I'm an assistant. I don't have a personal name as I'm a computer program, but you can call me Assistant. How can I assist you today?

Second interaction:


Parent run df919f89-3c64-4309-8719-a5fd81489ff9 not found for run 9139a873-70f7-44d5-ae0c-46826d340109. Treating as a root run.


Assistant:  Hello Bob! Nice to meet you too. I can tell you that Python is a high-level, interpreted programming language that was created by Guido van Rossum and first released in 1991. It is designed with an emphasis on code readability, and its syntax allows programmers to express concepts in fewer lines of code than would be possible in languages such as C++ or Java.

Python supports multiple programming paradigms, including procedural, object-oriented, and functional programming. It is commonly used for web development, data analysis, artificial intelligence, machine learning, and scientific computing, among other applications.

Python has a large standard library that includes areas like internet protocols, string operations, web services tools, and operating system interfaces. Many high use programming tasks are already scripted into the standard library which reduces length of code to be written significantly.

Python is also known for its strong support for integration with ot

In [16]:
with open('../output1.txt', 'r') as f:
    required_prompt = f.readlines()

In [17]:
type(required_prompt)

list

In [18]:
required_prompt[:10]

["vram\\EmulatorPkg\\Unix\\lldbinit:imports-['command script import lldbefi.py']\n",
 'vram\\MdeModulePkg\\Library\\BrotliCustomDecompressLib\\brotli\\configure-cmake:imports-[\'#   You must include the GNUInstallDirs module (which comes with\', \'#   CMake) in your project.  Just put "include (GNUInstallDirs)" in\']\n',
 "vram\\MdeModulePkg\\Library\\BrotliCustomDecompressLib\\brotli\\Makefile:imports-['$(CC) $(CFLAGS) $(CPPFLAGS) -Ic/include \\\\']\n",
 'vram\\MdeModulePkg\\Library\\BrotliCustomDecompressLib\\brotli\\python\\Makefile:imports-[\'EXT_SUFFIX=$(shell $(PYTHON) -c \\\'import sysconfig; print(sysconfig.get_config_var("EXT_SUFFIX"))\\\')\']\n',
 'vram\\MdeModulePkg\\Universal\\RegularExpressionDxe\\oniguruma\\INSTALL:imports-[\'below.  The lack of an optional feature in a given package is not\', \'The simplest way to compile this package is:\', "\'./configure\' to configure the package for your system.", \'recommended that the package be configured and built as a regular\',

In [19]:
required_prompt = "".join(required_prompt[:30])

In [20]:
print(len(required_prompt))

8514


Feed this into a conversational chain

In [27]:
print(f"Model: {model_selected}")

# Chat history management
store = {}  # memory is maintained outside the chain

def get_session_history(session_id: str) -> InMemoryChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

def print_chat_history(session_id: str):
    history = get_session_history(session_id)
    print(f"\nChat History for session {session_id}:")
    for message in history.messages:
        print(f"{message.type}: {message.content}")

# Create a proper LLM object
llm = ChatOpenAI(
    base_url='https://opensource-challenger-api.prdlvgpu1.aiaccel.dell.com/v1',
    model=model_selected,
    openai_api_key=os.environ['LLM_API'],
    http_client=http_client
)

chain = RunnableWithMessageHistory(llm, get_session_history)

# Check initial state
print_chat_history("1")

# First interaction
print("\nFirst interaction:")
response1 = chain.invoke(
    required_prompt+"Understand the codebase and the project structure.",
    config={"configurable": {"session_id": "1"}},
)
print("Assistant:", response1.content)

# Check history after first interaction
print_chat_history("1")

# Second interaction
print("\nSecond interaction:")
response2 = chain.invoke(
    "Can you generate test automation scripts on this project given the information or do you need more?",
    config={"configurable": {"session_id": "1"}},
)
print("Assistant:", response2.content)

# Check history after second interaction
print_chat_history("1")



Model: mixtral-8x7b-instruct-v01


Parent run e1666d88-233b-492c-8be1-b742ab26a60d not found for run 5e888432-2367-470f-b70e-e89906f26dc0. Treating as a root run.



Chat History for session 1:

First interaction:


Parent run 04d9e985-0978-4fa0-8c85-937bcadd5eca not found for run 1abbd9e5-6275-4201-99c0-2485c9d0e699. Treating as a root run.


Assistant:  The text you provided appears to be fragments of various configuration and documentation files from a codebase. I will attempt to summarize the information contained in these fragments.

1. lldbinit: This file is related to the LLDB debugger and imports a Python module `lldbefi.py`.
2. configure-cmake: This file is a CMake configuration script for the BrotliCustomDecompressLib library and mentions the GNUInstallDirs module.
3. Makefile: This is a Makefile for the BrotliCustomDecompressLib library, using the C compiler and including headers from the `c/include` directory.
4. python/Makefile: This is a Makefile for building Python extensions in the BrotliCustomDecompressLib library, extracting the extension suffix from the Python configuration.
5. INSTALL: This is a generic installation instruction file for the oniguruma package, mentioning various ways to configure and compile the package for different systems and environments.
6. depcomp: This file is an Autoconf script for

It kinda works I think the prompts need to be more streamlined and then I can work on the post processing and preprocessing of the file

In [5]:
with open('../output1.txt', 'r') as f:
    content = f.readlines()

store = {}  # memory is maintained outside the chain

def get_session_history(session_id: str) -> InMemoryChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

llm = ChatOpenAI(
    base_url='https://opensource-challenger-api.prdlvgpu1.aiaccel.dell.com/v1',
    model=model_selected,
    openai_api_key=os.environ['LLM_API'],
    http_client=http_client,
    stream_usage=True
)

chain = RunnableWithMessageHistory(llm, get_session_history)


current_context = "".join(content[:10])
ctr = 10
llm_request = "need more"
while llm_request == "need more":
    print(f"Current Context: {len(current_context)}")
    response = chain.invoke(
        current_context+" write code for test automation of the given context or say 'need more' to request for more information.",
        config={"configurable": {"session_id": "1"}},
    )
    print("Assistant:", response.content)
    if 'need' in response.content.lower() or 'more' in response.content.lower():
        llm_request = "need more"
        ctr+=10
        current_context = "".join(content[ctr:ctr+10]) if ctr+10 < len(content) else "".join(content[ctr:])
    else:
        llm_request = "done"



Parent run 65e10a2c-5954-4693-be9f-cc6160c6bffe not found for run 22038ca2-c20f-4971-8682-967f8d9c85fe. Treating as a root run.


Current Context: 944


Parent run 68e22393-079d-4aab-ada5-c4c62a59e855 not found for run 9392d4d2-8e22-4230-93ca-7b3368078d00. Treating as a root run.


Assistant:  In order to write test automation code for the given context, I would need to have a better understanding of what is being tested and the desired behavior of the libraries mentioned. However, I can provide some general guidance on how to approach writing test automation for these libraries.

Firstly, it is important to note that these libraries are part of the UEFI (Unified Extensible Firmware Interface) framework, which is a modern firmware interface specification for computers. Therefore, the test automation code will likely need to be written in a language that can interface with UEFI, such as C or C++.

Here are some specific steps that could be taken to write test automation for the given libraries:

1. **Set up a testing framework:** There are several testing frameworks available for UEFI, such as UEFITest and MiniYAMLTest. These frameworks can help simplify the process of writing and running tests for UEFI drivers and libraries.
2. **Understand the library interfaces

Parent run ba26fabf-37d3-4a90-ada8-b2fc3da69ef6 not found for run 50af3f9b-c0d7-4f49-b537-0de79091bbfd. Treating as a root run.


Assistant:  To write test automation code for the given context, we would need to first identify the specific functionality within these libraries that we want to test. Here are some potential test cases based on the information provided:

1. **Test SerialPortLib functions:** We could write test cases for the serial I/O port functions provided by `SerialPortLib`. For example, we could write a test case that opens a serial port, writes some data to it, and then reads the data back to verify that it was transmitted correctly.
2. **Test S3StallLib behavior:** We could write test cases for the `S3StallLib` library to verify its behavior during S3 resume. For example, we could write a test case that puts the system into S3 sleep, waits for a specified amount of time, and then verifies that the system resumes correctly.
3. **Test SmmServicesTableLib functions:** We could write test cases for the `SmmServicesTableLib` library to verify its functionality. For example, we could write a test cas